In [3]:
from selenium import webdriver
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup
from pymongo import MongoClient
from selenium.webdriver import Firefox
from selenium.webdriver import FirefoxOptions
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import pandas as pd

import glob
import json
import time
import re
import os

In [2]:
pip install webdriver_manager

Note: you may need to restart the kernel to use updated packages.


In [5]:
# Using Firefox
search_term = 'laptop'
base_url='https://www.amazon.com'
#url = f"{base_url}/s?k={search_term}"
url='https://www.amazon.com/s?k=laptop'

executable_path='/Users/sheetalrajgure/Desktop/MSBA Coursework/Winter Quarter/Machine Learning/GroupProject/DDR_part/geckodriver.exe'
options = Options()
options.headless = True
driver = webdriver.Firefox(executable_path=executable_path,options=options) #,options=options

driver.implicitly_wait(10)
driver.set_script_timeout(120)
driver.set_page_load_timeout(30)
#load
driver.get(url)
time.sleep(5)

page_source = driver.page_source

# Parse the HTML content of the page using BeautifulSoup
soup = BeautifulSoup(page_source, "html.parser")

# Find the "Highly rated" section of the page
highly_rated = soup.find("div", {"class": "s-include-content-margin s-border-bottom s-border-top-overlap s-widget-padding-bottom"})
highly_rated_data=highly_rated.find('span',{'data-component-type':'s-searchgrid-carousel'})

laptops_1 = highly_rated.find_all("li", {"class": "a-carousel-card"}) #backup is laptops variable

laptops_info = []
for laptop in laptops_1:
    a_tag = laptop.find("a", {"class": "a-link-normal s-no-outline"})
    if a_tag:
        laptop_url = a_tag.get("href")
        if laptop_url.startswith('/'):
            laptop_url = urljoin(base_url, laptop_url)
        laptops_info.append(laptop_url)
        print(f"Added laptop {laptop_url} to laptops_info.")
    else:
        print("No a_tag found for laptop.")
    

for i, product_url in enumerate(laptops_info):
    try:
        driver.get(product_url)  
        time.sleep(10)
        # Save the HTML content of the response to a local file
        filename = f"product_{i+1}.htm"
        with open(filename, "wb") as f:
            f.write(driver.page_source.encode('utf-8'))
        print(f"Saved product {i+1}")
    except Exception as e:
        print(f"Error occurred while processing product {i+1}: {str(e)}")


/var/folders/y5/6g4z_69535j4kbmdvkmpf9y40000gn/T/ipykernel_5472/3843004555.py:9: DeprecationWarning: headless property is deprecated, instead use add_argument('-headless')
  options.headless = True
/var/folders/y5/6g4z_69535j4kbmdvkmpf9y40000gn/T/ipykernel_5472/3843004555.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path=executable_path,options=options) #,options=options


Added laptop https://www.amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_search_thematic_aps_sr_pg1_1?ie=UTF8&adId=A01924322DGINHM0P6EE6&qualifier=1678680197&id=3868834816707174&widgetName=sp_search_thematic&url=%2FHP-Pavilion-Processor-Conferencing-Bluetooth%2Fdp%2FB09BBHXYZ9%2Fref%3Dsxin_14_pa_sp_search_thematic_sspa%3Fcontent-id%3Damzn1.sym.db5bdf6b-c3cb-4492-ada6-1426f4f364d5%253Aamzn1.sym.db5bdf6b-c3cb-4492-ada6-1426f4f364d5%26cv_ct_cx%3Dlaptop%26keywords%3Dlaptop%26pd_rd_i%3DB09BBHXYZ9%26pd_rd_r%3De04647f6-681d-4e89-93f2-c85912961ee3%26pd_rd_w%3DHPdMc%26pd_rd_wg%3DuiZs1%26pf_rd_p%3Ddb5bdf6b-c3cb-4492-ada6-1426f4f364d5%26pf_rd_r%3D7R4ASJJH6NFAQS7JJSZR%26qid%3D1678680197%26sbo%3DRZvfv%252F%252FHxDF%252BO5021pAnSA%253D%253D%26sr%3D1-1-a73d1c8c-2fd2-4f19-aa41-2df022bcb241-spons%26psc%3D1%26smid%3DA2HR3WADFB8IU9 to laptops_info.
Added laptop https://www.amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_search_thematic_aps_sr_pg1_2?ie=UTF8&adId=A08681383BVLXS3TB2LYR&quali

In [6]:
# Main Review URLs
review_urls=[]
for i, product_url in enumerate(laptops_info):
    try:
        driver.get(product_url)  
        time.sleep(10)
        # Save the HTML content of the response to a local file
        reviews_source=driver.page_source
        reviews_soup = BeautifulSoup(reviews_source, "html.parser")
        rev=reviews_soup.find('a',{'data-hook':'see-all-reviews-link-foot'})['href']
        if rev.startswith('/'):
            rev2 = urljoin(base_url, rev)
        review_urls.append(rev2)
        print(review_urls)
    except Exception as e:
        print(f"Error occurred while processing product {i}")


['https://www.amazon.com/HP-Pavilion-Processor-Conferencing-Bluetooth/product-reviews/B09BBHXYZ9/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews']
['https://www.amazon.com/HP-Pavilion-Processor-Conferencing-Bluetooth/product-reviews/B09BBHXYZ9/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews', 'https://www.amazon.com/Acer-Swift-Quad-Core-i5-11300H-Fingerprint/product-reviews/B0BXRZNFQ9/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews']
['https://www.amazon.com/HP-Pavilion-Processor-Conferencing-Bluetooth/product-reviews/B09BBHXYZ9/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews', 'https://www.amazon.com/Acer-Swift-Quad-Core-i5-11300H-Fingerprint/product-reviews/B0BXRZNFQ9/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews', 'https://www.amazon.com/HP-Display-Processor-Graphics-Microfiber/product-reviews/B0B1VZFZ6R/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews']
['https://www.amazon.com/HP-Pavilion-Processor-Conf

In [7]:

# create a folder to store the HTML files
if not os.path.exists('html_pages'):
    os.makedirs('html_pages')


# scrape all review pages for each main review URL and save as HTML files
for i, rev_url in enumerate(review_urls):
    j = 1
    product = f'laptop_{i+1}'
    while True:
        try:
            url = f'{rev_url}&pageNumber={j}'
            driver.get(url)
            time.sleep(10)
            reviews_source = driver.page_source
            with open(f'html_pages/{product}_page{j}.html', 'w', encoding='utf-8') as f:
                f.write(reviews_source)
            reviews_soup = BeautifulSoup(reviews_source, "html.parser")
            reviews = reviews_soup.find_all('div', {'id': 'cm_cr-review_list'})
            if not reviews or not reviews[0].find_all('div', {'data-hook': 'review'}):
                break
            j += 1
        except Exception as e:
            print(f"Error occurred while processing product {i+1}, page {j}")
            break


In [10]:

reviews = []

for file_name in os.listdir('html_pages'):
    # extract product name from file name
    product = file_name.split('_page')[0]
    
    # parse HTML content using BeautifulSoup
    with open(f'html_pages/{file_name}', 'r', encoding='utf-8') as f:
        page_source = f.read()
    soup = BeautifulSoup(page_source, 'html.parser')
    
    # extract reviews and ratings
    review_boxes = soup.find_all('div', {'data-hook': 'review'})
    for i, box in enumerate(review_boxes):
        review = f'review_{i+1}'
        rating_element = box.find('i', {'data-hook': 'review-star-rating'})
        if rating_element:
            rating = rating_element.text.strip().split(' ')[0]
        else:
            rating = None
        text_element = box.find('span', {'data-hook': 'review-body'})
        if text_element:
            text = text_element.text.strip()
        else:
            text = None
        reviews.append({'product': product, 'review': review, 'rating': rating, 'text': text})



df = pd.DataFrame(reviews, columns=['product', 'review', 'rating', 'text'])


In [11]:
df

,product,review,rating,text
0,laptop_3,review_1,5.0,I purchased this to use for my family research...
1,laptop_3,review_2,2.0,"Much to my surprise, I opened the box and saw ..."
2,laptop_3,review_3,1.0,I bought two of these laptops in August 2022 f...
3,laptop_3,review_4,2.0,The colors on the screen are really bad and it...
4,laptop_3,review_5,2.0,I just received this laptop within the week of...
...,...,...,...,...
239,laptop_3,review_6,5.0,I got this during a prime day sale and I'm gla...
240,laptop_3,review_7,1.0,This was the worse product purchased ever! The...
241,laptop_3,review_8,3.0,
242,laptop_3,review_9,3.0,"Acer should not use the MediaTek Wifi card, it..."



# DONT USE THIS
reviews_data = []

for i, rev_url in enumerate(review_urls):
    j = 1
    product=f'laptop_{i+1}'
    while True:
        try:
            url = f'{rev_url}&pageNumber={j}'
            driver.get(url)
            time.sleep(10)
            reviews_source = driver.page_source
            reviews_soup = BeautifulSoup(reviews_source, "html.parser")
            reviews = reviews_soup.find('div', {'id': 'cm_cr-review_list'})
            if not reviews:
                break
            for review in reviews:
                rating = review.find('i', {'data-hook': 'review-star-rating'}).text.split()[0]
                text = review.find('span', {'data-hook': 'review-body'}).text
                reviews_data.append({'Product': product, 'Rating': rating, 'Review': text})
            j += 1
        except Exception as e:
            print(f"Error occurred while processing product {i}, page {j}")
            break
print(reviews_data)

# DONT USE THIS
## Using Firefox
search_term = 'laptop'
base_url='https://www.amazon.com'
url = f"{base_url}/s?k={search_term}"


executable_path='/Users/aravindsampath/Desktop/Winter Quarter/BAX 422 Data Design & Representation/Assignments/Final Project/geckodriver'
options = Options()
options.headless = True
driver = webdriver.Firefox(executable_path=executable_path,options=options) #,options=options


# DONT USE THIS
### Using Safari webdriver
driver = webdriver.Safari()

### Load the URL directly
driver.get("https://www.amazon.com/s?k=laptop")

### Wait for the page to load
time.sleep(5)

### Find the "Amazon's Choice" section of the page
try:
    amazons_choice_section = driver.find_element_by_xpath("//span[contains(text(), 'Amazon’s Choice')]")
    print(amazons_choice_section.text)
except:
    print("Amazon's Choice not found on the page")
    
### Close the browser
#driver.quit()
